In [1]:
import numpy as np
import random
import torch
print(torch.__version__)

1.12.0+cu113


/home/sz/anaconda3/envs/cling/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_everything(11) 

In [3]:
torch.randn((3,5))

tensor([[ 0.7376,  1.9459, -0.6995, -1.3023, -0.5133],
        [-0.2696,  0.2462,  0.4839,  0.4504, -0.9568],
        [ 1.5012, -0.3136, -0.2343, -1.0713,  0.1648]])

In [4]:
new_x = torch.randn((4,3))
print(new_x)

tensor([[ 0.1157,  0.3834, -0.1234],
        [ 0.8929, -0.4891, -1.4470],
        [ 1.1941,  0.5462, -2.0750],
        [-0.9691, -2.1354, -1.0431]])


In [5]:
class MyModule(torch.nn.Module):
    def __init__(self,N, M):
        super(MyModule, self).__init__()
        self.linear1 = torch.nn.Linear(N, M)
        self.linear2 = torch.nn.Linear(M, 1)

    def forward(self, input):
        out0 = self.linear1(input)
        print(out0)
        out0_relu = torch.nn.functional.relu(out0)
        return self.linear2(out0_relu)

model = MyModule(2,3)
print(model.state_dict())
print(dict(model.state_dict()))

OrderedDict([('linear1.weight', tensor([[-0.3501,  0.0384],
        [ 0.4640, -0.0062],
        [ 0.4655, -0.6389]])), ('linear1.bias', tensor([-0.5903, -0.3211,  0.3810])), ('linear2.weight', tensor([[ 0.3484, -0.4291,  0.2168]])), ('linear2.bias', tensor([-0.1036]))])
{'linear1.weight': tensor([[-0.3501,  0.0384],
        [ 0.4640, -0.0062],
        [ 0.4655, -0.6389]]), 'linear1.bias': tensor([-0.5903, -0.3211,  0.3810]), 'linear2.weight': tensor([[ 0.3484, -0.4291,  0.2168]]), 'linear2.bias': tensor([-0.1036])}


In [6]:
x = torch.ones(4,2)
print(x)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])


In [7]:
output = model(x)
print(output)

tensor([[-0.9019,  0.1367,  0.2076],
        [-0.9019,  0.1367,  0.2076],
        [-0.9019,  0.1367,  0.2076],
        [-0.9019,  0.1367,  0.2076]], grad_fn=<AddmmBackward0>)
tensor([[-0.1173],
        [-0.1173],
        [-0.1173],
        [-0.1173]], grad_fn=<AddmmBackward0>)


In [8]:
torch.save(dict(model.state_dict()), 'none_embeded_model_state_dict.pt')

In [9]:
class EmbedModule(torch.nn.Module):
    def __init__(self):
        super(EmbedModule, self).__init__()
        self.linear0 = torch.nn.Linear(3, 2)
        self.model = MyModule(2,3)
        
    def forward(self, input):
        output1 = self.linear0(input)
        output2 = self.model(output1)
        return output2

In [10]:
embeded_model = EmbedModule()
print(embeded_model.state_dict())

OrderedDict([('linear0.weight', tensor([[ 0.1673,  0.5430, -0.5589],
        [ 0.1020,  0.3467,  0.0483]])), ('linear0.bias', tensor([-0.3457, -0.1063])), ('model.linear1.weight', tensor([[-0.2108,  0.5481],
        [ 0.6317, -0.2776],
        [-0.6945, -0.2240]])), ('model.linear1.bias', tensor([-0.3902,  0.6022,  0.6675])), ('model.linear2.weight', tensor([[ 0.2981, -0.0493, -0.5566]])), ('model.linear2.bias', tensor([0.0415]))])


In [11]:
output = embeded_model(new_x)
print(output)

tensor([[-0.3621,  0.5622,  0.6944],
        [-0.6029,  0.8921,  0.4836],
        [-0.6091,  1.4010, -0.2661],
        [-0.7075,  0.1938,  1.6436]], grad_fn=<AddmmBackward0>)
tensor([[-0.3727],
        [-0.2716],
        [-0.0275],
        [-0.8829]], grad_fn=<AddmmBackward0>)


In [12]:
#print(list(embeded_model.children()))

In [13]:
torch.save(dict(embeded_model.state_dict()), 'embeded_model_state_dict.pt')